### Setup

In [1]:
import os, sys, importlib, time, inspect
from dotenv import load_dotenv

# Ensure repo root is first on sys.path
repo_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if sys.path[0] != repo_root:
    if repo_root in sys.path:
        sys.path.remove(repo_root)
    sys.path.insert(0, repo_root)

# Ensure RAG and AB Testing directories follow the repo root
rag_dir = os.path.join(repo_root, 'RAG')
if rag_dir in sys.path:
    sys.path.remove(rag_dir)
sys.path.insert(1, rag_dir)

test_dir = os.path.join(repo_root, 'AB Testing')
if test_dir in sys.path:
    sys.path.remove(test_dir)
sys.path.insert(2, test_dir)

# Drop any previously loaded RAG modules to avoid stale code
for name in list(sys.modules):
    if name == 'RAG' or name.startswith('RAG.'):
        del sys.modules[name]

# Load .env and configure timezone
env_path = os.path.join(repo_root, '.env')
load_dotenv(env_path, override=True)

timezone = os.getenv('TZ', 'America/New_York')
os.environ['TZ'] = timezone
time.tzset()

current_dir = os.getcwd()
print(f"Current directory: {current_dir}")
print(f"Parent directory: {repo_root}")
print(f"Parent dir exists: {os.path.exists(repo_root)}")
print(f"RAG dir exists: {os.path.exists(rag_dir)}")
print(f"KnowledgeGraph exists: {os.path.exists(os.path.join(repo_root, 'KnowledgeGraph'))}")

# Import modules after paths are set up
from KnowledgeGraph.config import load_neo4j_graph
import RAG.Citations as citations_module
import RAG.VectorRAG as VectorRAG
import RAG.GraphRAG as GraphRAG
import RAG.batch_query as batch_query
import mark_answers as mark_answers_module

# Reload modules to pick up latest code
importlib.reload(citations_module)
importlib.reload(VectorRAG)
importlib.reload(GraphRAG)
importlib.reload(batch_query)
importlib.reload(mark_answers_module)

mark_answers = mark_answers_module.mark_answers
from RAG.GraphRAG import query_graph_rag

print("format_with_citations signature:", inspect.signature(citations_module.format_with_citations))

graph, openAI_api, openAI_endpoint, openAI_model = load_neo4j_graph()


Current directory: /mnt/c/Users/sepeh/OneDrive/Documents/Git/GraphRag_Developer_Challenge3/Main Functions
Parent directory: /mnt/c/Users/sepeh/OneDrive/Documents/Git/GraphRag_Developer_Challenge3
Parent dir exists: True
RAG dir exists: True
KnowledgeGraph exists: True


/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


format_with_citations signature: (answer: 'str', references: 'Sequence[Reference]', style: 'str' = 'vancouver', seq_offset: 'int' = 0) -> 'str'


/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/neo4j/_sync/driver.py:547: DeprecationWarning: Relying on Driver's destructor to close the session is deprecated. Please make sure to close the session. Use it as a context (`with` statement) or make sure to call `.close()` explicitly. Future versions of the driver will not close drivers automatically.
  _deprecation_warn(


### GraphRag Single Query

In [2]:
catalog_question = "For tender 2021/2020/06, what are the closing date, closing time, bid validity, tender document fee, whether alternative offers are allowed, and whether the tender is indivisible?"
supplement_question = "What is the company name listed for Tender RFP-2113556?"
spotlight_question = "Across both the late-April Amiri Diwan tender and Practice A/M/804, are alternative offers allowed?"
spotlight_question2 = "What is the practice fee for Ministry of Health Practice No. 5DX003?" # It should say 75 KD [1]\n\nReferences\n 1. 2025-05-04_en.md. p.29.

GraphRAG.query_graph_rag(spotlight_question2)

'75 KD [1]\n\nReferences\n1. 71-1737-0.'

### GraphRAG Batch Query

In [3]:
qp1 = "AB Testing/questions_long.json"
qp2 = "AB Testing/questions_supplement.json"
qp3 = "AB Testing/questions_short.json"
qp4 = "AB Testing/questions_unseen.json"

input_question_file = qp1

batch_query.batch_query_graph_rag(input_question_file=input_question_file, max_workers=20, show_progress=True)

Processed 71 queries with 12 workers                                            
Results saved to: /mnt/c/Users/sepeh/OneDrive/Documents/Git/GraphRag_Developer_Challenge3/AB Testing/Output Answers/answers_20251103_011326.json


[{'question': 'What is the time period to submit an objection for company registration changes?',
  'answer': '- Most company registration changes (e.g., legal-entity amendments, dissolution/liquidation): 30 days from the date of publication to file an objection with the General Court (المحكمة الكلية).\n- Converting an individual license to a company at the Companies Department: 15 days from the date of publication to submit a written objection to the department, usually with a debt instrument (proof of indebtedness).',
  'events': [{'event': 'gate',
    'run_id': 'q0',
    'question': 'What is the time period to submit an objection for company registration changes?',
    'anchors': [],
    'matched_ids': [],
    'decision': 'pass',
    'reason': 'no_match'},
   {'event': 'retrieval',
    'run_id': 'q0',
    'pool_top_k': 300,
    'hybrid_retrieved': 300,
    'ranker': 'linear',
    'alpha': 0.6,
    'fulltext_sanitized': True,
    'vector_fallback': False,
    'scope_applied': False,


### Mark Answers

In [4]:
if input_question_file == qp1:
    answer_key_path = "AB Testing/answer_key_long.json"
elif input_question_file == qp2:
    answer_key_path = "AB Testing/answer_key_supplement.json" 
elif input_question_file == qp3:
    answer_key_path = "AB Testing/answer_key_short.json"
elif input_question_file == qp4:
    answer_key_path = "AB Testing/answer_key_unseen.json"
else:
    input_question_file = 0

if input_question_file != 0:
    mark_answers(answer_key_path=answer_key_path, max_workers=60, show_progress=True)
else:
    print("No question file selected; skipping.")


TEST RESULTS

Configuration:
  top_k: 120
  MMR_k: 60
  MMR_LAMBDA: 0.4
  ALWAYS_KEEP_TOP: 10
  neighbor_window: 1

Results:
  Average F1: 56.0% (0.560)

Card-first gate:
  CARD_FIRST_ENABLED: True
  ALLOW_DIRECT_ANSWER: True

